# Feature selection

## Data from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

PATH = '/content/gdrive/MyDrive/Creditcard_data/creditcard.csv';

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Credit card data

In [2]:
import pandas as pd

creditcard_df = pd.read_csv(PATH)
creditcard_df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
creditcard_df.shape

(284807, 31)

## Class-imbalance - fraud transactions

In [4]:
creditcard_df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

## Test Training Dataset

In [5]:
from sklearn.model_selection import train_test_split

X = creditcard_df.drop(['Class'],axis=1)
y = creditcard_df.Class

## RFECV

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFECV

rfe = RFECV(estimator=GradientBoostingClassifier(random_state=0),min_features_to_select=5, step=10)

X_rfe = pd.DataFrame(rfe.fit_transform(X, y))
X_rfe.columns = rfe.get_feature_names_out()

X_train, X_test, y_train, y_test = train_test_split(X_rfe,y,random_state=0, test_size=0.33)

print("train rows: {}, test rows: {}".format(X_train.shape[0], X_test.shape[0]))  # rows

### Decision tree clssifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)

dt.score(X_test, y_test)

0.9992445763775841

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  123,    39],
       [   32, 93793]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.92%
sensitivity = 75.93%
specificity = 99.97%



### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

rf.score(X_test, y_test)

0.999521210380159

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  125,    37],
       [    8, 93817]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.95%
sensitivity = 77.16%
specificity = 99.99%



### MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)

mlp.score(X_test, y_test)

0.9993616138402119

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  125,    37],
       [   23, 93802]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.94%
sensitivity = 77.16%
specificity = 99.98%



## Forward Logistic Regression

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

flr = SequentialFeatureSelector(LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000), n_features_to_select=5, direction='forward')

X_flr = pd.DataFrame(flr.fit_transform(X, y))
X_flr.columns = flr.get_feature_names_out()

X_train, X_test, y_train, y_test = train_test_split(X_flr,y,random_state=0, test_size=0.33)

print("train rows: {}, test rows: {}".format(X_train.shape[0], X_test.shape[0]))  # rows

train rows: 190820, test rows: 93987


### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

rf.score(X_test, y_test)

0.9995531296881484

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  127,    35],
       [    7, 93818]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.96%
sensitivity = 78.40%
specificity = 99.99%



### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)

dt.score(X_test, y_test)

0.9991062593762967

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  115,    47],
       [   37, 93788]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.91%
sensitivity = 70.99%
specificity = 99.96%



### MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)

mlp.score(X_test, y_test)

0.9992764956855735

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[   98,    64],
       [    4, 93821]])

In [ ]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.93%
sensitivity = 60.49%
specificity = 100.00%



## PCA

In [7]:
from sklearn.decomposition import PCA

pca = PCA()
X_pca = pd.DataFrame(pca.fit_transform(X, y))

X_train, X_test, y_train, y_test = train_test_split(X_pca,y,random_state=0, test_size=0.33)

print("train rows: {}, test rows: {}".format(X_train.shape[0], X_test.shape[0]))  # rows

train rows: 190820, test rows: 93987


### Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)

dt.score(X_test, y_test)

0.9990637002989775

In [9]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  112,    50],
       [   38, 93787]])

In [10]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.91%
sensitivity = 69.14%
specificity = 99.96%



### Random forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

rf.score(X_test, y_test)

0.9995318501494888

In [12]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[  126,    36],
       [    8, 93817]])

In [13]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.95%
sensitivity = 77.78%
specificity = 99.99%



### MLP

In [14]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train,y_train)

y_pred = mlp.predict(X_test)

mlp.score(X_test, y_test)

0.9988189856043921

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels=[1,0])

array([[   73,    89],
       [   22, 93803]])

In [16]:
from sklearn.metrics import accuracy_score


tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[1,0]).ravel()
specificity = tn / (tn + fp) * 100
sensitivity = tp / (tp + fn) * 100
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"accuracy = {accuracy:.2f}%\nsensitivity = {sensitivity:.2f}%\nspecificity = {specificity:.2f}%\n")

accuracy = 99.88%
sensitivity = 45.06%
specificity = 99.98%



## Conclusion

For none of the algorithms: decision tree, random forest and mlp did not improve the sensitivity of the model using feature selection